In [1]:

import numpy as np 

In [2]:
def quat2rot(q):
    R = np.array([[q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2, - 2*(q[1] * q[2] - q[0] * q[3]),  - 2*(q[1] * q[3] + q[0] * q[2])],
                  [- 2*(q[1] * q[2] + q[0] * q[3]),         
                   q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2, 2*(q[2] * q[3] - q[0] * q[1])],
                  [- 2*(q[1] * q[3] - q[0] * q[2]),         2*(q[2] * q[3] + q[0] * q[1]),    q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2]])
    return R
                  

def quat2zyz(q):
    r32 =  2*(q[2] * q[3] - q[0] * q[1])
    r31 =  2*(q[1] * q[3] + q[0] * q[2])
    r33 =  q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2
    r13 =  2*(q[1] * q[3] - q[0] * q[2])
    r23 =  2*(q[2] * q[3] + q[0] * q[1])
   
    x = np.arctan2(r32, r31)
    z = np.arctan2(r23, -r13)
    y = np.arctan2(r31*np.cos(x) + r32*np.sin(x), r33)
      #  y  = np.arccos(r13)
    return np.array([z,y,x])

def quaternion_to_euler(q):
    (w, x, y, z) = (q[0], q[1], q[2], q[3])
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll = np.arctan2(t0, t1)
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch = np.arcsin(t2)
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw = np.arctan2(t3, t4)
    return np.array([roll, pitch, yaw])

def quatmult(x,y):
    return np.array([x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
                     x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
                     x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
                     x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]])

def euler_to_quaternion(r):
    (yaw, pitch, roll) = (r[0], r[1], r[2])
    qz = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qx = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return np.array([qw, qx, qy, qz])

In [3]:
def r_x(x): 
    return np.array([[1,0,0], [0, np.cos(x), - np.sin(x)], [0, np. sin(x), np.cos(x)]])
def r_y(x): 
    return np.array( [[np.cos(x), 0, np.sin(x)], [0,1,0], [-np. sin(x), 0 ,  np.cos(x)]])
def r_z(x): 
    return np.array([[np.cos(x), - np.sin(x), 0], [np. sin(x), np.cos(x),0], [0,0,1]])


def xyz2quat(x,y,z):
    qx = np.array([np.cos(x/2), np.sin(x/2), 0, 0])
    qy = np.array([np.cos(y/2), 0, np.sin(y/2), 0])
    qz = np.array([np.cos(z/2), 0, 0, np.sin(z/2)])
    return quatmult(qx, quatmult(qy, qz))

def zyz2quat(x,y,z):
    qx = np.array([np.cos(x/2), 0, 0, np.sin(x/2)])
    qy = np.array([np.cos(y/2), 0, np.sin(y/2), 0])
    qz = np.array([np.cos(z/2), 0, 0, np.sin(z/2)])
    return quatmult(qz, quatmult(qy, qx))

def zyz2xyz(x,y,z):
    R = r_z(x) @ r_y(y) @ r_z(z)
    x = np.arctan(- R[1,2]/R[2,2])
    y = np.arcsin(R[0,2])
    z = np.arctan(-R[0,1] /R[0,0])
    return x,y,z

def xyz2zyz(e):
    R = r_x(e[0]) @ r_y(e[1]) @ r_z(e[2])
    if R[2,2] != 0:
        y = np.arccos(R[2,2])
        z = -np.arctan2(R[2,1], R[2,0]) +np.pi
        x = np.arctan2(R[1,2], R[0,2])
        
    return np.array([x,y,z])

In [4]:
# transforms points to quaterniions and back
x = 0.0
y = 0.3 
z = 1.3
xyz2quat(*zyz2xyz(x,y,z)) #- euler_to_quaternion(zyz2xyz(x,y,z))

array([0.78714464, 0.09043793, 0.11896528, 0.59839081])

In [5]:
#define transformation from H to R³
## works

def MRP_H2R(q):
    return 4 * q[1:]/(1+q[0])

def MRP_R2H(e):
    return 1/(16 + np.linalg.norm(e)**2) * np.array([16 - np.linalg.norm(e)**2, *8*e])

MRP_H2R(np.array([1,0,0,0]))

array([0., 0., 0.])

In [6]:
# complex conjugated 
def quat_cc(q):
    d = np.copy(q)
    d[1:] *= -1
    return d

In [7]:
x = 0
y = 0.5 
z = 1
a = euler_to_quaternion(zyz2xyz(x,y,z))

In [8]:
quat2rot(a).T

array([[ 0.47415988, -0.73846026,  0.47942554],
       [ 0.84147098,  0.54030231,  0.        ],
       [-0.25903472,  0.40342268,  0.87758256]])

In [9]:
from numpy import sin, cos
def r_z_r_y_r_z(ret, v):
    ret[0,0] = -sin(v[0])*sin(v[2]) + cos(v[0])*cos(v[1])*cos(v[2])
    ret[0,1] = sin(v[0])*cos(v[2]) + sin(v[2])*cos(v[0])*cos(v[1])
    ret[0,2] = - sin(v[1])*cos(v[0])
    ret[1,0] = - sin(v[0])*cos(v[1])*cos(v[2]) - sin(v[2])*cos(v[0])
    ret[1,1] = - sin(v[0])*sin(v[2])*cos(v[1]) + cos(v[0])*cos(v[2])
    ret[1,2] = sin(v[0])*sin(v[1])
    ret[2,0] = sin(v[1])*cos(v[2])
    ret[2,1] = sin(v[1])*sin(v[2])
    ret[2,2] = cos(v[1])
    

In [10]:
R = np.zeros((3,3))
r_z_r_y_r_z(R, [0,0.5,1])

In [11]:
R

array([[ 0.47415988,  0.73846026, -0.47942554],
       [-0.84147098,  0.54030231,  0.        ],
       [ 0.25903472,  0.40342268,  0.87758256]])

In [12]:
R = np.zeros((3,3))
r_z_r_y_r_z(R, [0,0.5,1])

In [13]:
R

array([[ 0.47415988,  0.73846026, -0.47942554],
       [-0.84147098,  0.54030231,  0.        ],
       [ 0.25903472,  0.40342268,  0.87758256]])

In [14]:
def rotation_angles(matrix, order='zyz'):
    """
    input
        matrix = 3x3 rotation matrix (numpy array)
        oreder(str) = rotation order of x, y, z : e.g, rotation XZY -- 'xzy'
    output
        theta1, theta2, theta3 = rotation angles in rotation order
    """
    a = np.eye(3)
    a[0,0] = -1
    matrix = a @ matrix @ a
    r11, r12, r13 = matrix[0]
    r21, r22, r23 = matrix[1]
    r31, r32, r33 = matrix[2]
   

    if order == 'xzx':
        theta1 = np.arctan(r31 / r21)
        theta2 = np.arctan(r21 / (r11 * np.cos(theta1)))
        theta3 = np.arctan(-r13 / r12)

    elif order == 'xyx':
        theta1 = np.arctan(-r21 / r31)
        theta2 = np.arctan(-r31 / (r11 *np.cos(theta1)))
        theta3 = np.arctan(r12 / r13)

    elif order == 'yxy':
        theta1 = np.arctan(r12 / r32)
        theta2 = np.arctan(r32 / (r22 *np.cos(theta1)))
        theta3 = np.arctan(-r21 / r23)

    elif order == 'yzy':
        theta1 = np.arctan(-r32 / r12)
        theta2 = np.arctan(-r12 / (r22 *np.cos(theta1)))
        theta3 = np.arctan(r23 / r21)

    elif order == 'zyz':
        theta1 = np.arctan(r23 / r13)
        theta2 = np.arctan(r13 / (r33 *np.cos(theta1)))
        theta3 = np.arctan(-r32 / r31)

    elif order == 'zxz':
        theta1 = np.arctan(-r13 / r23)
        theta2 = np.arctan(r23 / (r33 *np.cos(theta1)))
        theta3 = np.arctan(r31 / r32)

    elif order == 'xzy':
        theta1 = np.arctan(r32 / r22)
        theta2 = np.arctan(-r12 * np.cos(theta1) / r22)
        theta3 = np.arctan(r13 / r11)

    elif order == 'xyz':
        theta1 = np.arctan(-r23 / r33)
        theta2 = np.arctan(r13 * np.cos(theta1) / r33)
        theta3 = np.arctan(-r12 / r11)

    elif order == 'yxz':
        theta1 = np.arctan(r13 / r33)
        theta2 = np.arctan(-r23 * np.cos(theta1) / r33)
        theta3 = np.arctan(r21 / r22)

    elif order == 'yzx':
        theta1 = np.arctan(-r31 / r11)
        theta2 = np.arctan(r21 * np.cos(theta1) / r11)
        theta3 = np.arctan(-r23 / r22)

    elif order == 'zyx':
        theta1 = np.arctan(r21 / r11)
        theta2 = np.arctan(-r31 * np.cos(theta1) / r11)
        theta3 = np.arctan(r32 / r33)

    elif order == 'zxy':
        theta1 = np.arctan(-r12 / r22)
        theta2 = np.arctan(r32 * np.cos(theta1) / r22)
        theta3 = np.arctan(-r31 / r33)

    theta1 = theta1
    theta2 = theta2
    theta3 = theta3
    
    return np.array((theta1, theta2, theta3))

import numpy as np

def rotation_matrix(theta1, theta2, theta3, order='zyz'):
    """
    input
        theta1, theta2, theta3 = rotation angles in rotation order (degrees)
        oreder = rotation order of x,y,z　e.g. XZY rotation -- 'xzy'
    output
        3x3 rotation matrix (numpy array)
    """
    c1 = np.cos(theta1)
    s1 = np.sin(theta1)
    c2 = np.cos(theta2)
    s2 = np.sin(theta2)
    c3 = np.cos(theta3)
    s3 = np.sin(theta3)

    if order == 'xzx':
        matrix=np.array([[c2, -c3*s2, s2*s3],
                         [c1*s2, c1*c2*c3-s1*s3, -c3*s1-c1*c2*s3],
                         [s1*s2, c1*s3+c2*c3*s1, c1*c3-c2*s1*s3]])
    elif order=='xyx':
        matrix=np.array([[c2, s2*s3, c3*s2],
                         [s1*s2, c1*c3-c2*s1*s3, -c1*s3-c2*c3*s1],
                         [-c1*s2, c3*s1+c1*c2*s3, c1*c2*c3-s1*s3]])
    elif order=='yxy':
        matrix=np.array([[c1*c3-c2*s1*s3, s1*s2, c1*s3+c2*c3*s1],
                         [s2*s3, c2, -c3*s2],
                         [-c3*s1-c1*c2*s3, c1*s2, c1*c2*c3-s1*s3]])
    elif order=='yzy':
        matrix=np.array([[c1*c2*c3-s1*s3, -c1*s2, c3*s1+c1*c2*s3],
                         [c3*s2, c2, s2*s3],
                         [-c1*s3-c2*c3*s1, s1*s2, c1*c3-c2*s1*s3]])
    elif order=='zyz':
        matrix=np.array([[c1*c2*c3-s1*s3, c3*s1+c1*c2*s3, -c1*s2],
                         [- c1*s3-c2*c3*s1, c1*c3-c2*s1*s3, s1*s2],
                         [c3*s2, s2*s3, c2]])
    elif order=='zxz':
        matrix=np.array([[c1*c3-c2*s1*s3, -c1*s3-c2*c3*s1, s1*s2],
                         [c3*s1+c1*c2*s3, c1*c2*c3-s1*s3, -c1*s2],
                         [s2*s3, c3*s2, c2]])
    elif order=='xyz':
        matrix=np.array([[c2*c3, -c2*s3, s2],
                         [c1*s3+c3*s1*s2, c1*c3-s1*s2*s3, -c2*s1],
                         [s1*s3-c1*c3*s2, c3*s1+c1*s2*s3, c1*c2]])
    elif order=='xzy':
        matrix=np.array([[c2*c3, -s2, c2*s3],
                         [s1*s3+c1*c3*s2, c1*c2, c1*s2*s3-c3*s1],
                         [c3*s1*s2-c1*s3, c2*s1, c1*c3+s1*s2*s3]])
    elif order=='yxz':
        matrix=np.array([[c1*c3+s1*s2*s3, c3*s1*s2-c1*s3, c2*s1],
                         [c2*s3, c2*c3, -s2],
                         [c1*s2*s3-c3*s1, c1*c3*s2+s1*s3, c1*c2]])
    elif order=='yzx':
        matrix=np.array([[c1*c2, s1*s3-c1*c3*s2, c3*s1+c1*s2*s3],
                         [s2, c2*c3, -c2*s3],
                         [-c2*s1, c1*s3+c3*s1*s2, c1*c3-s1*s2*s3]])
    elif order=='zyx':
        matrix=np.array([[c1*c2, c1*s2*s3-c3*s1, s1*s3+c1*c3*s2],
                         [c2*s1, c1*c3+s1*s2*s3, c3*s1*s2-c1*s3],
                         [-s2, c2*s3, c2*c3]])
    elif order=='zxy':
        matrix=np.array([[c1*c3-s1*s2*s3, -c2*s1, c1*s3+c3*s1*s2],
                         [c3*s1+c1*s2*s3, c1*c2, s1*s3-c1*c3*s2],
                         [-c2*s3, s2, c2*c3]])
    return matrix

In [15]:
import bonndit.utilc.quaternions as qu
import matplotlib.pyplot as plt
import numpy as np

In [17]:
a = np.zeros((4))
a = np.random.random(size=4)
ret = np.zeros((3))
qu.MPR_H2R_q(ret, a,a, 1, 1)

In [18]:
b = np.random.normal(0,0.05,(3, 20))

In [20]:
c = np.zeros((4,20))
d = np.zeros((3,3,20))
e = np.zeros((3,3))
for i in range(20):
    qu.MPR_R2H_q(c[:,i], b[:,i], a, 1,1)

In [22]:
for i in range(20):
    qu.quat2rot(d[:,:,i], c[:,i])

In [23]:
from scipy.spatial.transform import Rotation as R
r = R.from_euler(seq='zyz', angles=[0, 0, np.pi/2])

In [24]:
r.as_matrix()

array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [25]:
x = np.random.random(3)
quat2rot(xyz2quat(*zyz2xyz(*x)))

array([[ 0.18321394,  0.92849104, -0.32301245],
       [-0.92307904,  0.27550789,  0.26836632],
       [ 0.3381682 ,  0.24899757,  0.90754751]])

In [29]:
for i in range(100):
    x = np.random.random(3) * 10*np.pi -5*np.pi
    r = np.zeros(4)
    u =np.zeros((3))
    # sanity check zyz to quat
    qu.ZYZ2quat(r, x)
    r1 = R.from_euler(seq='zyz', angles=x)
    print(r - np.roll(r1.as_quat(),1))
    # sanity check quat to zyz
    h =np.zeros((3))
    qu.quat2ZYZ(h, r)

  #  h = quat2zyz(r)
    print(np.round(r1.as_euler('zyz')- h, 5) )
    # sanity check quat to rot: 
    h =np.zeros((3,3))
    h1 =np.zeros((3,3))
    r_z_r_y_r_z(h, x)
    qu.quat2rot(h1, r)
    print(np.round(r1.as_matrix() - h1,4))
    # R mapping check:
    ret = np.zeros((3))
    qu.MPR_H2R_q(ret, r, r, 1, 1)
    print(ret)
    r2 = np.zeros((4))
    qu.MPR_R2H_q(r2, ret, r, 1, 1)
    print(r-r2)
    r3 = np.random.random(4)
    r3 *= 1/np.linalg.norm(r3)
    ret = np.zeros((3))
    qu.MPR_H2R_q(ret, r, r3, 1, 1)
    r2 = np.zeros((4))
    qu.MPR_R2H_q(r2, ret, r3, 1, 1)
    print(np.round(r2-r,4))
    

 
    

[0. 0. 0. 0.]
[0. 0. 0.]
[[-0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[-0. -0.  0.  0.]
[0. 0. 0. 0.]
[ 0. -0.  0.]
[[-0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[-0. -0.  0.  0.]
[0. 0. 0. 0.]
[0. 0. 0.]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[ 0. -0.  0.  0.]
[0. 0. 0. 0.]
[0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0.]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[-0.  0. -0.  0.]
[0. 0. 0. 0.]
[ 0. -0.  0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[ 0. -0. -0. -0.]
[0. 0. 0. 0.]
[0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[-0.  0.  0.  0.]
[0. 0. 0. 0.]
[0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0.]
[0. 0. 0. 0.]
[ 0. -0.  0.  0.]
[0. 0. 0. 0.]
[0. 0. 

In [24]:
import nrrd
import nibabel as nib 
watson_file, b = nrrd.read('/mnt/uni/HCPTestFolder/922854/infolder/watson.nrrd')
fodf = nib.load('/mnt/uni/HCPTestFolder/922854/infolder/fodf.nii.gz')
fodf = fodf.get_fdata()


In [25]:
import numpy as np
dj_o4 = np.array([
    [[1.0,0.0, -0.5, -0.0,0.375],
    [0.0, -0.70710678, -0.0,0.4330127,0.],
    [0.0,0.0,0.61237244,0.0, -0.39528471],
    [0.0,0.0,0.0, -0.55901699, -0.],
    [0.0,0.0,0.0,0.0,0.52291252]
    ],
    [[0.0,0.70710678,0.0, -0.4330127, -0.],
    [0.0,0.5, -0.5, -0.125,0.375],
    [0.0,0.0, -0.5,0.39528471,0.1767767],
    [0.0,0.0,0.0,0.48412292, -0.33071891],
    [0.0,0.0,0.0,0.0, -0.46770717]
    ],
    [[0.0,0.0,0.61237244,0.0, -0.39528471],
    [0.0,0.0,0.5, -0.39528471, -0.1767767],
    [0.0,0.0,0.25, -0.5,0.25],
    [0.0,0.0,0.0, -0.30618622,0.46770717],
    [0.0,0.0,0.0,0.0,0.33071891]
    ],
    [[0.0,0.0,0.0,0.55901699,0.],
    [0.0,0.0,0.0,0.48412292, -0.33071891],
    [0.0,0.0,0.0,0.30618622, -0.46770717],
    [0.0,0.0,0.0,0.125, -0.375],
    [0.0,0.0,0.0,0.0, -0.1767767]
    ],
    [[0.0,0.0,0.0,0.0,0.52291252],
    [0.0,0.0,0.0,0.0,0.46770717],
    [0.0,0.0,0.0,0.0,0.33071891],
    [0.0,0.0,0.0,0.0,0.1767767],
    [0.0,0.0,0.0,0.0,0.0625]]])

In [31]:
import bonndit.utilc.watsonfitwrapper as w
from bonndit.utilc.cython_helpers import cart2sphere, r_z_r_y_r_z

In [27]:
## Watson reference:

In [28]:
x = 83
y = 109
z = 100
d = watson_file[2:,0,x,y,z]
d[0] *= -1
d[2] *= -1
d = d.astype(np.float64)
s = np.zeros(3, dtype=np.float64)
cart2sphere(s[1:], d)
s[0] = -2


In [29]:
kappa = watson_file[0,0,x,y,z]
l = watson_file[1,0,x,y,z]
v = np.zeros((15), dtype=np.float64)
w.p_sh_watson_coeffs(kappa, v, 4)


In [30]:
v[0] *=2.51327412
v[3] *=  1.43615664
v[10] *= 0.31914592

In [31]:
v

array([0.70898154, 0.        , 0.        , 0.81405978, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.18965412, 0.        , 0.        , 0.        , 0.        ])

In [32]:
pysh = np.zeros(50, dtype=np.float64)
w.p_map_dipy_to_pysh_o4(v, pysh)

In [33]:
rot_pysh = np.zeros(50, dtype=np.float64)
w.p_sh_rotate_real_coef(rot_pysh, pysh, 4, s, dj_o4)

In [34]:
v2  = np.zeros((15), dtype=np.float64)
w.p_map_pysh_to_dipy_o4(rot_pysh, v2)

In [35]:
np.linalg.norm(v2 * l - fodf[x,y,z,:])

0.44857523197934224

In [32]:
#Now bingham:

import bonndit.utilc.quaternions as qu

In [37]:
b = np.load('./src/bonndit/tracking/kalman/bingham_coefs_versuch1001.npy')
n = np.load('./src/bonndit/tracking/kalman/normalize_const_versuch1000.npy')

In [38]:
for i,j in np.ndindex(b.shape[:2]):
    if n[i,j] != 0:
        b[i,j] *= 1/n[i,j]
    else:
        b[i,j] = 0

In [39]:
b[...,0,:] *= 2.51327412
b[...,1,:] *= 1.43615664
b[...,2,:] *= 0.31914592

In [40]:
quater = np.zeros(4, dtype=np.float64)
qu.ZYZ2quat(quater, s)
mean = np.array([l, min(kappa, 50), 0.2, *quater])

In [41]:
kappa

15.383438

In [42]:
angels = np.zeros(3, dtype=np.float64)
kappa=mean[1]
beta=mean[2]
qu.quat2ZYZ(angels, mean[3:])
dipy_v = np.zeros(15, dtype=np.float64)
dipy_v[0] = b[int(kappa*10),  int(beta*10), 0, 0]
dipy_v[1] = b[int(kappa*10),  int(beta*10), 1, 2]
dipy_v[2] = b[int(kappa*10),  int(beta*10), 1, 1]
dipy_v[3] = b[int(kappa*10),  int(beta*10), 1, 0]
dipy_v[4] = b[int(kappa*10),  int(beta*10), 1, 8]
dipy_v[5] = b[int(kappa*10),  int(beta*10), 1, 7]
dipy_v[6] = b[int(kappa*10),  int(beta*10), 2, 4]
dipy_v[7] = b[int(kappa*10),  int(beta*10), 2, 3]
dipy_v[8] = b[int(kappa*10),  int(beta*10), 2, 2]
dipy_v[9] = b[int(kappa*10),  int(beta*10), 2, 1]
dipy_v[10] = b[int(kappa*10),  int(beta*10), 2, 0]
dipy_v[11] = b[int(kappa*10),  int(beta*10), 2, 8]
dipy_v[12] = b[int(kappa*10),  int(beta*10), 2, 7]
dipy_v[13] = b[int(kappa*10),  int(beta*10), 2, 6]
dipy_v[14] = b[int(kappa*10),  int(beta*10), 2, 5]


In [43]:
pysh = np.zeros(50, dtype=np.float64)
w.p_map_dipy_to_pysh_o4(dipy_v, pysh)
rot_pysh = np.zeros(50, dtype=np.float64)
w.p_sh_rotate_real_coef(rot_pysh, pysh, 4, s, dj_o4)
v2  = np.zeros((15), dtype=np.float64)
w.p_map_pysh_to_dipy_o4(rot_pysh, v2)
np.linalg.norm(v2 * l - fodf[x,y,z,:])

0.44844375278566123

In [34]:
import numpy as np
for i in range(100):
    d = np.random.random(3) *10 -20
    s = np.zeros(3, dtype=np.float64)
    cart2sphere(s[1:], d)
    s[0] = -2
    quater = np.zeros(4, dtype=np.float64)
    qu.ZYZ2quat(quater, s)
    R = np.zeros((3,3), dtype=np.float64)
    qu.quat2rot(R, quater)
    #print(np.round(R, 4))
    R2 = np.zeros((3,3), dtype=np.float64)
    r_z_r_y_r_z(R2, s)    
    print(np.round(R2-R.T,4))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 0. -0.  0.]
 [-0.  0.  0.]
 [ 0.  0. -0.]]
[[-0. -0.  0.]
 [ 0. -0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0.  0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0.  0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 0.  0.  0.]
 [ 0.  0. -0.]
 [ 0.  0. -0.]]
[[ 0. -0.  0.]
 [-0. -0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0.  0.]
 [-0. -0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0.  0.]
 [-0. -0.  0.]
 [ 0.  0.  0.]]
[[-0. -0.  0.]
 [-0.  0. -0.]
 [ 0. -0. -0.]]
[[ 0. -0.  0.]
 [-0. -0.  0.]
 [ 0.  0.  0.]]
[[ 0.  0.  0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0. -0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0.  0.]
 [-0. -0.  0.]
 [ 0.  0.  0.]]
[[-0.  0.  0.]
 [-0.  0. -0.]
 [-0. -0. -0.]]
[[ 0. -0.  0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0. -0.]
 [ 0. -0.  0.]
 [ 0.  0.  0.]]
[[ 0. -0. -0.]
 [ 0.  0.  0.]
 [ 0.  0. -0.]]
[[ 0. -0.  0.]
 [-0.  0.  0.]
 [ 0. -0. -0.]]
[[ 0.  0.  0.]
 [-0.  0. -0.]
 [-0. -0.  0.]]
[[ 0. -0.  0.]
 [-0. -0.  0.]
 [ 0.  0.  0.]]
[[ 0. 

array([[ 0.23156693,  0.86145553, -0.45196364],
       [ 0.94554749, -0.09007537,  0.31277207],
       [ 0.22872844, -0.49978075, -0.83540559]])

0.0758594

In [4]:
pip install .

Processing /home/johannes/UniBonn/forschung/bonndit
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.24.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Created wheel for bonndit: filename=bonndit-0.2.0-cp38-cp38-linux_x86_64.whl size=30458170 sha256=dc5b8355ce98b66ac8f35f77ae03e052c0acf90880bcd7fd4b67395983f9c3b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-de5vkzbk/wheels/2e/1e/92/44625976e847c3f0ac3a02bc8b14a2722cbba93db73d0e6491
Successfully built bonndit
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.0
    Uninstalling numpy-1.20.0:
      Successfully uninstalled numpy-1.20.0
  Attempting uninstall: bonndit
    Found existing installation: bonndit 0.2.0
    Uninstalling bonndit-0.2.0:
      Successfully uninstalled bonndit-0.2.0

[notice] A new release of pip is available: 23.0.

In [5]:
pip uninstall numpy -y

Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy==1.20.0

  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyshtools 4.10.1 requires numpy>=1.23.2, but you have numpy 1.20.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /home/johannes/UniBonn/forschung/bonndit/venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
